In [2]:
import numpy as np

embeddings_names = [
    "polblogs",
    "cs_phds",
    "cora",
    "citeseer",
]

# Copied from notebook 2
SIGNATURES = [
    [(-1, 2), (-1, 2)],  # HH
    [(-1, 2), (0, 2)],  # HE
    [(-1, 2), (1, 2)],  # HS
    [(1, 2), (1, 2)],  # SS
    [(1, 2), (0, 2)],  # SE
    [(-1, 4)],  # H
    [(0, 4)],  # E
    [(1, 4)],  # S
]
SIGNATURES_STR = ["HH", "HE", "HS", "SS", "SE", "H", "E", "S"]
n_trials = 10

bad = []
for embedding in embeddings_names:
    for signature in SIGNATURES_STR:
        for trial in range(n_trials):
            my_data = np.load(f"../data/graphs/embeddings/{embedding}/{signature}_{trial}.npy")
            if np.isnan(my_data).any():
                bad.append((embedding, signature, trial))
                print(embedding, signature, trial)
print(bad)

[]


In [6]:
pd.read_table(f"../data/graphs/embeddings/{embedding}_dists.tsv")["signature"]

0     [(-1, 2), (-1, 2)]
1     [(-1, 2), (-1, 2)]
2     [(-1, 2), (-1, 2)]
3     [(-1, 2), (-1, 2)]
4     [(-1, 2), (-1, 2)]
             ...        
75              [(1, 4)]
76              [(1, 4)]
77              [(1, 4)]
78              [(1, 4)]
79              [(1, 4)]
Name: signature, Length: 80, dtype: object

In [3]:
import embedders
import pandas as pd
from tqdm.notebook import tqdm

results = []
my_tqdm = tqdm(total=len(embeddings_names) * n_trials * len(SIGNATURES))
for embedding in embeddings_names:
    _, y, _ = embedders.dataloaders.load(embedding)
    dists = pd.read_table(f"../data/graphs/embeddings/{embedding}_dists.tsv")
    for sig, sig_str in zip(SIGNATURES, SIGNATURES_STR):
        pm = embedders.manifolds.ProductManifold(signature=sig)
        for trial in range(n_trials):
            X = np.load(f"../data/graphs/embeddings/{embedding}/{sig_str}_{trial}.npy")

            res = embedders.benchmarks.benchmark(
                X,
                y,
                pm,
                # models=["sklearn_dt", "product_dt"],
                max_depth=3,
                n_features="d_choose_2"
            )
            res["embedding"] = embedding
            res["signature"] = sig_str
            res["trial"] = trial
            res["d_avg"] = dists[]

            results.append(res)
            my_tqdm.update(1)

results = pd.DataFrame(results)

  0%|          | 0/320 [00:00<?, ?it/s]

Top CC has 1222 nodes; original graph has 1490 nodes.
Top CC has 1025 nodes; original graph has 1025 nodes.
Top CC has 2485 nodes; original graph has 2708 nodes.
Top CC has 2110 nodes; original graph has 3264 nodes.


In [4]:
# results.to_csv("../data/results/graphs.tsv", sep="\t", index=False)
results.to_csv("../data/results/graphs_dC2.tsv", sep="\t", index=False)